In [0]:
from pyspark.sql.functions import *
import urllib

In [0]:
file_type = "delta"

first_row_is_header = "true"

delimiter = ","

aws_keys_df = spark.read.format(file_type)\
    .option("header", first_row_is_header)\
    .option("sep", delimiter)\
    .load("dbfs:/user/hive/warehouse/pranavarora_99_access_keys")

In [0]:
ACCESS_KEY = aws_keys_df.select('Access key ID').take(1)[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').take(1)[0]['Secret access key']

In [0]:
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
AWS_S3_BUCKET = "pranavdataproject1"

MOUNT_NAME = "/mnt/pranavdataproject1"

SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)

In [0]:
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)

In [0]:
display(dbutils.fs.ls("/mnt/pranavdataproject1/amazon"))

In [0]:
file_location = "/mnt/pranavdataproject1/amazon/Air Conditioners.csv"
file_type = "csv"

infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

df = spark.read.format(file_type)\
    .option("inferSchema", infer_schema)\
    .option("sep", delimiter)\
    .load(file_location)
display(df)


In [0]:
from pyspark.sql.functions import monotonically_increasing_id

In [0]:
df = spark.read.format("csv").option("header", True).load(
    "/mnt/pranavdataproject1/amazon/Air Conditioners.csv"
)


In [0]:
df = df.withColumn("ID", monotonically_increasing_id())

In [0]:
df.write.format("csv").option("header", True).save("/mnt/pranavdataproject1/amazon/updated_Air Conditioners.csv")

In [0]:
display(df)

In [0]:
options = {
    "sfURL": "https://jjwxott-xwb11429.snowflakecomputing.com",
    "sfUSER": "",
    "sfPassword": "",
    "sfDatabase": "amazondb",
    "sfSchema": "SCHEMA",
    "sfWarehouse": "COMPUTE_WH",
}


In [0]:
df = spark.read.format("csv").option("header", "true").load(
    "/mnt/pranavdataproject1/amazon/updated_Air Conditioners.csv").write.format("snowflake").options(**options).mode("append").option("dbtable", "transformed_amazon_table").save()

In [0]:
df = spark.read\
    .format("snowflake")\
        .options(**options)\
            .option("query", "select * from transformed_amazon_table;")\
                .load()

display(df)